# MediGenie: Medical ChatBot AI


## Overview
MediGenie is an AI-driven chatbot designed to assist with medical queries by leveraging state-of-the-art language models and biomedical tools. The project integrates advanced AI capabilities with specialized APIs such as NCBI Entrez to offer insights, retrieve information, and provide intelligent responses tailored to healthcare needs.


## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
   - [Importing Libraries](#importing-libraries)
   - [Loading Environment Variables](#loading-environment-variables)
3. [Model Initialization](#model-initialization)
4. [NCBI Entrez API Configuration](#ncbi-entrez-api-configuration)
5. [Usage](#usage)
6. [Future Enhancements](#future-enhancements)

## 1. Introduction
MediGenie leverages the power of advanced artificial intelligence, particularly in the fields of natural language processing (NLP) and machine learning, to bridge the gap between medical professionals and essential healthcare information. By analyzing vast amounts of biomedical data, research articles, clinical guidelines, and patient records, MediGenie empowers users to obtain accurate and relevant medical insights quickly.

Designed with both medical professionals and patients in mind, MediGenie offers a user-friendly interface that allows for natural language-based queries, enabling users to ask medical questions in everyday language. Whether it's symptom identification, treatment recommendations, or understanding complex medical conditions, MediGenie provides real-time, evidence-backed responses that support clinical decision-making and patient care.

With its ability to process large datasets, MediGenie also aids in identifying patterns and trends, helping healthcare providers make data-driven decisions. Furthermore, its AI capabilities evolve and improve over time, continuously learning from new research, medical publications, and user interactions to enhance its accuracy and effectiveness.

Ultimately, MediGenie strives to improve health outcomes by facilitating timely access to trustworthy medical information, enhancing the decision-making process, and empowering users with knowledge for better healthcare management.

## 2. Setup
### Installing Dependencies

In [ ]:
pip install langchain langchain-groq python-dotenv biopython requests

### Importing Libraries

In [ ]:
# Import necessary libraries
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from dotenv import load_dotenv
from Bio import Entrez
import os

These libraries include:
- **LangChain**: For agent-based operations and tool integrations.
- **ChatGroq**: For conversational AI models.
- **Bio.Entrez**: For accessing NCBI Entrez.
- **dotenv**: For managing environment variables.


## 3. Loading Environment Variables and Model Initialization 

In [ ]:
# Load Environment Variables
load_dotenv()
groq_api_key = 'Your Groq API KEY'

# Initialize Chat Model
llm = ChatGroq(api_key=groq_api_key, model="llama-3.3-70b-versatile")


Explanation:
- **load_dotenv()**: Loads variables from a `.env` file.
- **groq_api_key**: Stores the API key for ChatGroq.
- The `ChatGroq` model is used to power conversational capabilities.
- The API key ensures secure access to the model.


## 4. NCBI Entrez API Configuration

In [ ]:
# Set email for NCBI Entrez API (required for proper usage)
Entrez.email = "Your Mail ID"

## 5. Usage
To interact with the chatbot, initialize tools and execute queries using LangChain’s agent framework:

In [ ]:
# Function to retrieve articles from PubMed
def retrieve_pubmed_articles(query, max_results=5):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    
    id_list = record["IdList"]
    if not id_list:
        return []
    
    handle = Entrez.efetch(db="pubmed", id=",".join(id_list), rettype="medline", retmode="text")
    articles = handle.read()
    handle.close()
    
    return articles

# Mock Medical Model Function
def mock_medical_model(symptom: str) -> str:
    # Simulate a response based on the symptom
    responses = {
        "headache": "Headaches can be caused by various factors including tension, dehydration, or migraines. It's advisable to stay hydrated and rest.",
        "fever": "Fever is often a sign of infection. Common causes include flu or other viral infections. Monitor your symptoms and consult a doctor if necessary.",
        "cough": "A cough can indicate a cold, flu, or allergies. If persistent, consider seeing a healthcare provider.",
        "nausea": "Nausea can be due to food poisoning, motion sickness, or pregnancy. If it persists, consult a healthcare professional."
    }
    return responses.get(symptom.lower(), "Consult a healthcare provider for more information.")


In [ ]:
# Define Tools for Medical Queries
@tool
def symptom_mapping(symptom: str) -> str:
    """
    Maps symptoms to possible medical conditions.
    
    Args:
    symptom (str): A comma-separated string of symptoms to analyze.
    
    Returns:
    str: A list of possible conditions mapped to the symptoms.
    """
    mappings = {
        "fever": "Possible flu or infection",
        "headache": "Could be a migraine or dehydration",
        "cough": "Possibly a cold, flu, or COVID-19",
        "nausea": "May indicate food poisoning or early pregnancy"
    }
    mapped = [mappings.get(s.strip().lower(), "Unknown") for s in symptom.split(",")]
    return ", ".join(mapped)

@tool
def medication_advice(condition: str) -> str:
    """
    Provides medication advice based on the medical condition.
    
    Args:
    condition (str): The medical condition to provide advice for.
    
    Returns:
    str: Medication advice for the specified condition.
    """
    advice = {
        "fever": "Use paracetamol or ibuprofen, and stay hydrated.",
        "migraine": "Consider ibuprofen or a prescribed migraine medication.",
        "cold": "Rest, stay hydrated, and consider antihistamines."
    }
    return advice.get(condition.lower(), "Consult a doctor for advice.")

@tool
def first_aid_guidance(issue: str) -> str:
    """
    Provides first aid guidance for specific health issues.
    
    Args:
    issue (str): The health issue to get first aid guidance for.
    
    Returns:
    str: First aid guidance for the specified health issue.
    """
    first_aid = {
        "burn": "Cool the burn under running water for 20 minutes. Do not apply ice.",
        "cut": "Clean the wound and apply pressure to stop bleeding.",
        "choking": "Perform the Heimlich maneuver. Call emergency services if needed."
    }
    return first_aid.get(issue.lower(), "Refer to a certified first aid guide.")

@tool
def dietary_recommendation(condition: str) -> str:
    """
    Provides dietary recommendations based on medical conditions.
    
    Args:
    condition (str): The medical condition to provide dietary advice for.
    
    Returns:
    str: Dietary recommendations for the specified condition.
    """
    diet_plans = {
        "diabetes": "Focus on low-glycemic foods, vegetables, and whole grains.",
        "hypertension": "Limit sodium intake and include potassium-rich foods.",
        "anemia": "Increase iron intake through spinach, legumes, and red meat."
    }
    return diet_plans.get(condition.lower(), "Consult a dietitian for tailored advice.")

@tool
def retrieve_pubmed_articles(symptom: str) -> str:
    """
    Retrieves relevant PubMed articles for the given symptom.
    
    Args:
    symptom (str): The symptom to search PubMed for.
    
    Returns:
    str: A list of relevant PubMed articles.
    """
    # This is a mock function, replace it with actual PubMed API retrieval logic.
    return f"Mock articles related to {symptom}."

@tool
def mock_medical_model(symptom: str) -> str:
    """
    Provides medical details about the symptom using a mock medical model.
    
    Args:
    symptom (str): The symptom to analyze.
    
    Returns:
    str: Mock medical information related to the symptom.
    """
    # This is a mock function, replace it with actual medical model logic.
    return f"Mock medical details for {symptom}."

@tool
def enhanced_medical_query(symptom: str) -> str:
    """
    Provides a detailed medical query response including possible conditions, medical details, and relevant articles.
    
    Args:
    symptom (str): The symptom to analyze.
    
    Returns:
    str: Detailed medical information including possible conditions, medical details, and PubMed articles.
    """
    # Use the symptom mapping to get possible conditions
    possible_conditions = symptom_mapping.invoke(symptom)
    
    # Retrieve articles from PubMed for more detailed information (Mocked)
    pubmed_articles = retrieve_pubmed_articles.invoke(symptom)
    # Use the mock medical model to provide a specific answer (Mocked)
    medical_answer = mock_medical_model.invoke(symptom)
    
    return f"Possible Conditions: {possible_conditions}\nMedical Details: {medical_answer}\nPubMed Articles: {pubmed_articles}"

# Aggregate Tools
medical_tools = [
    symptom_mapping,
    medication_advice,
    first_aid_guidance,
    dietary_recommendation,
    enhanced_medical_query
]


In [ ]:
class MediGenie:
    def __init__(self, tools, llm):
        self.agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        self.state = {}  # Track symptoms discussed
        self.termination_triggered = False

    def ask_clarifying_questions(self, symptom):
        clarifications = {
            "dizziness": "Could you describe when the dizziness occurs (e.g., after standing up, while lying down)?",
            "headache": "Where is the pain located? Is it sharp, dull, or throbbing?",
            "cough": "Is the cough dry or productive? Are there additional symptoms like fever?"
        }
        return clarifications.get(symptom.lower(), f"Could you tell me more about your symptom '{symptom}'?")

    def handle_input(self, user_input):
        # Basic input validation
        if not user_input.strip():
            return "I didn't catch that. Could you describe your symptom in more detail?"
    
        if user_input in self.state:  # Prevent repetitive discussions
            return f"It seems we've already discussed {user_input}. Any new symptoms or questions?"
    
        self.state[user_input] = True  # Mark the symptom as discussed
    
        try:
            response = self.agent.run(user_input)  # Process input
        except Exception as e:
            return f"Something went wrong while analyzing your symptom. Error: {str(e)}"
    
        # Ask for more information or feedback
        feedback_question = self.ask_clarifying_questions(user_input)
        return f"{response}\n\n{feedback_question}"

    def reset_session(self):
        self.state = {}
        self.termination_triggered = False
        return "The session has been reset. You can now start a new query."

    def check_termination(self):
     if self.termination_triggered:
        return "Thank you for using MediGenie. Before we close, do you have any more questions?"

    def terminate(self):
        self.termination_triggered = True
        return (
            "Thanks for using MediGenie! Remember to consult a healthcare provider for serious concerns. "
            "For emergencies, please contact local medical services immediately."
        )



## 6. Interactive ChatBot

In [ ]:
# Initialize LangChain Agent (only if required explicitly in the flow)
agent = initialize_agent(
    tools=medical_tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
 # Interactive session for MediGenie
def interactive_session(medi_genie):
    print("Welcome to MediGenie! Type 'exit' to end the session or 'reset' to start over.")
    while True:
        user_input = input("Describe your symptoms or ask a question: ")
        if user_input.lower() == 'exit':
            print(medi_genie.terminate())
            break
        elif user_input.lower() == 'reset':
            print(medi_genie.reset_session())
        else:
            response = medi_genie.handle_input(user_input)
            print("\nMediGenie Response:", response)

# Initialize MediGenie and start the session
medi_genie = MediGenie(tools=medical_tools, llm=llm)
interactive_session(medi_genie)


In [ ]:
# Test Case : Termination
print(medi_genie.terminate())
print(medi_genie.check_termination())